# Salto de potencial sob controle difusional 

Assumindo a transferência reversível de 1 elétron:
$$
\begin{equation*}
\text{O} + \text{ne}^- \rightleftharpoons \text{R}
\end{equation*}
$$

Assumindo eletrodo planar e considerando que as espécies tem o mesmo coeficiente de difusão, $ \text{para} \quad  0\le \tau \le \tau_s \quad 
  \text{e} \quad 0 \le y \le 6 \sqrt{D \tau_s}$:

\begin{align}
  \frac{\partial c_O}{\partial \tau} &= D \frac{\partial^2 c_O}{\partial y^2} \\
  \frac{\partial c_R}{\partial \tau} &= D \frac{\partial^2 c_R}{\partial y^2}
\end{align}

As condições inciais, assumindo R inicialmente ausente, são:
\begin{align}
C_O(y,0) &= C_O^* \\
C_R(y,0) &= 0
\end{align}

A difusão semi-infinita é aplicada no seio da solução:
\begin{align} 
\lim_{y \to \infty} C_O(y,\tau) &= C_O^* \\
\lim_{y \to \infty} C_R(y,\tau) &= 0
\end{align}

A condição de equilíbrio (equação de Nernst) é aplicada na interface:

\begin{equation}
\theta = \exp \left(\frac{F}{RT}(E-E^0) \right)
\end{equation}

\begin{align}
C_O(0,\tau) &= \frac{\theta}{1+\theta}  \quad \text{para} \, \tau>0 \\
C_R(0,\tau) &= \frac{1}{1+\theta} \quad \text{para} \, \tau>0
\end{align}

Definindo as seguintes transformações de variáveis:

\begin{equation*}
x=\frac{y}{\sqrt{D \tau_s}}
\end{equation*}

\begin{equation*}
t=\frac{\tau}{\tau_s}
\end{equation*}

\begin{equation*}
c_{O,R}=\frac{C_{O,R}}{C_O^*}
\end{equation*}

\begin{equation}
\eta = E-E^0
\end{equation}

\begin{equation}
f = \frac{F}{RT}
\end{equation}

As equações ficam, $ \quad \text {para} \quad 0 \le t \quad \text{e} \quad 0 \le x \le 6$:

\begin{align}
\tag{1.0a} 
\frac{\partial c_O}{\partial t} &=  \frac{\partial^2 c_O}{\partial x^2} \\
\tag{1.0b}
\frac{\partial c_R}{\partial t} &=  \frac{\partial^2 c_R}{\partial x^2}
\end{align}

E as condições iniciais e de contorno ficam:

\begin{align}
\tag{1.1a}
c_O(x,0)&=1 \\
\tag{1.1b}
c_R(x,0)&=0
\end{align}

\begin{align}
\tag{1.2a}
\lim_{x \to \infty} c_O(x,t) &= 1 \\
\tag{1.2b}
\lim_{x \to \infty} c_R(x,t) &= 0
\end{align}

\begin{align}
\theta &= \exp (f\eta) \\
\tag{1.3a}
c_O(0,t) &= \frac{\theta}{1+\theta} \\
\tag{1.3b}
c_R(0,t) &= \frac{1}{1+\theta}
\end{align}

A solução para a concentração é:
\begin{align}
c_O(x,t) &= 1- \frac{1}{1 + \theta}erfc \left ( \frac{x}{2\sqrt{t}} \right )\\
c_R(x,t) &= \frac{1}{1 + \theta}erfc \left ( \frac{x}{2\sqrt{t}} \right )
\end{align}

A densidade de corrente é limitada pelo fluxo difusional, logo:
\begin{equation*}
j(\tau) = nFD \left ( \frac{\partial C_O}{\partial y} \right )_{y=0} = nFD\frac{C_O^*}{\sqrt{D\tau}} \left ( \frac{\partial c}{\partial x} \right )_{x=0}
\end{equation*}

Definindo a corrente adimensional como $i(t)=\frac{j(\tau)}{nFD} \frac{\sqrt{D\tau}}{C_O^*}$ e derivando a expressão da concentração:
\begin{equation*}
\tag{3.0}
i(t)=\frac{1}{\sqrt{\pi t}}
\end{equation*}

Iniciamos importando as bibliotecas do pybamm e do python necessárias:


In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import math
from scipy import special

## Definindo o modelo

Definindo a variável do modelo e a qual domínio ela pertence (neste caso é necessário calcular a concentração de b para aplicar a equação de Nernst na interface):

In [2]:
model = pybamm.BaseModel()

co = pybamm.Variable("Concentration of O", domain="electrolyte") 
cr = pybamm.Variable("Concentration of R", domain="electrolyte")

Definindo os parâmetros do modelo:

In [3]:
eta = pybamm.Parameter("Applied Overpotential [V]") # sobrepotencial aplicado
F = pybamm.Parameter("Faraday constant [C.mol-1]")
R = pybamm.Parameter("Molar gas constant [J.K-1.mol-1]")
T = pybamm.Parameter("Temperature [K]")

As equações são definidas em termos do fluxo e do divergente e adicionadas ao dicionário `model.rhs`

In [4]:
No = - pybamm.grad(co)  # define o fluxo de O
Nr = -pybamm.grad(cr) # define o fluxo de R
rhso = -pybamm.div(No)  # define o lado direito da equação 1.0a
rhsr = -pybamm.div(Nr) # define o lado direito da equação 1.0b

model.rhs = {co: rhso, cr: rhsr}  # adicona as equações ao dicionário

Introduzindo as condições iniciais e de contorno:

In [5]:
# condições iniciais
model.initial_conditions = {co: pybamm.Scalar(1), cr:pybamm.Scalar(0)}

# boundary conditions
f = F/(R*T)
teta = pybamm.exp(f*eta)
left_co = teta/(1+teta) # equação 1.3a. Valor de co na interface
left_cr = 1/(1+teta) # equação 1.3b. Valor de cr na interface
right_co = pybamm.Scalar(1) # equação 1.2a. Valor de co no seio da solução.
right_cr = pybamm.Scalar(0) # equação 1.2b. Valor de cr no seio da solução.
model.boundary_conditions = {co: {"left": (left_co, "Dirichlet"), "right": (right_co, "Dirichlet")},
                             cr: {"left": (left_cr, "Dirichlet"), "right": (right_cr, "Dirichlet")}}
#Dirichlet refere-se à condições de contorno que determinam o valor das variáveis co e cr na fronteira.

Adicionando as variáveis de interesse ao dicionário `model.variables`. 

In [6]:
model.variables = {"Concentration of O": co, "Flux of O": No, "Concentration of R": cr, "Flux of R": Nr}

## Usando o modelo

### Definindo a geometria e a malha

As variáveis espaciais são definidas independentemente das variáveis do modelo. O domínio 1D varia no intervalo $0 \le x \le 6$. "eta" é deixado como input para permitir simulações com diferentes sobrepotenciais aplicados.

In [7]:
param = pybamm.ParameterValues(
    {
        "Applied Overpotential [V]": "[input]",
        "Faraday constant [C.mol-1]": 96485.3,
        "Molar gas constant [J.K-1.mol-1]": 8.31446,
        "Temperature [K]": 298.15
    }
)

In [8]:
# define geometry
x = pybamm.SpatialVariable(
    "x", domain=["electrolyte"], coord_sys="cartesian"
)
geometry = {"electrolyte": {x: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(6)}}}

In [9]:
param.process_model(model)
param.process_geometry(geometry)

Criando uma malha uniforme. A implementar : malha com expansão exponencial. (ver descrição abaixo)

In [10]:
# mesh and discretise
submesh_types = {"electrolyte": pybamm.Uniform1DSubMesh}
var_pts = {x: 400}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

Example of meshes that do require parameters include the `pybamm.Exponential1DSubMesh` which clusters points close to one or both boundaries using an exponential rule. It takes a parameter which sets how closely the points are clustered together, and also lets the users select the side on which more points should be clustered. For example, to create a mesh with more nodes clustered to the right (i.e. the surface in the particle problem), using a stretch factor of 2, we pass an instance of the exponential submesh class and a dictionary of parameters into the `MeshGenerator` class as follows: `pybamm.MeshGenerator(pybamm.Exponential1DSubMesh, submesh_params={"side": "right", "stretch": 2})`

Discretizando por Volumes Finito. A testar: Elementos finitos.

In [11]:
spatial_methods = {"electrolyte": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

### Resolvendo o modelo
Você pode obter soluções para valores inseridos para o sobrepotencial

O Solver ScipySolver é escolhido. Outras opções?. Definindo a malha uniforme no tempo. Como é feita a discretização no tempo? Como tratar stiff problems?  A implementar: comparação com soluções analiticas para corrente e concentração. 

In [12]:
from ipywidgets import interact, Text, Layout

# solução
solver = pybamm.ScipySolver()
t = np.linspace(0.00001, 1, 1000)
f = 38.9217


def plot_solution(eta_ap):
    try:
        eta_ap = float(eta_ap)  # Garante que a entrada seja um número válido.
        if eta_ap>0:
            print("Por favor, insira um sobrepotencial negativo")
            return
    except ValueError:
        print("Por favor, insira um número válido")
        return
    
    solution = solver.solve(model, t, inputs={"Applied Overpotential [V]": eta_ap})

    co_sol = solution["Concentration of O"]
    cr_sol = solution["Concentration of R"]
    No_sol = solution["Flux of O"]

    teta_ap = np.exp(f*eta_ap)

    def cr_an(t):
        return 1/(1+teta_ap)*special.erfc(x/(2*np.sqrt(t)))

    def i_an(t):
        return 1/(np.sqrt(np.pi*solution.t)*(1+teta_ap))

    
    # plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

    #theoretical and numerical concentration profile
    x = np.linspace(0, 6, 200)
    ax2.plot(x, cr_sol(t=1,x=x),"g.",x,cr_an(1),"g--",x,co_sol(t=1, x=x),"r.",x, 1-cr_an(1),"r--",
             x, cr_sol(t=0.1,x=x),"g.",x,cr_an(.1),"g--",x,co_sol(t=0.1, x=x),"r.",x, 1-cr_an(.1),"r--",
             x, cr_sol(t=0.01,x=x),"g.",x,cr_an(.01),"g--",x, co_sol(t=0.01, x=x),"r.",x, 1-cr_an(.01),"r--",
             x, cr_sol(t=0.001,x=x),"g.",x,cr_an(.001),"g--",x, co_sol(t=0.001, x=x),"r.",x, 1-cr_an(.001),"r--")
    ax2.set_xlabel("x")
    ax2.set_ylabel("Concentration")
    ax2.set_xlim([0,0.9])
    
    #theoretical and numerical current
    ax1.plot(solution.t,-No_sol(solution.t,x=0),"r .",solution.t,i_an(solution.t),"b")
    ax1.set_xlabel("t")
    ax1.set_ylabel("Current")
    ax1.set_xlim([0.01,1])
    ax1.set_ylim([0,6])
    plt.tight_layout()
    plt.show()

# Cria um Text widget para entrada do sobrepotencial
eta_text = Text(value='-0.1', description= 'entre $ \eta $ em V  ($\eta <0$):', continuous_update=False,
                style={'description_width':'initial'})

# Usa ipywidgets interact com o Text widget
interact(plot_solution, eta_ap=eta_text);


interactive(children=(Text(value='-0.1', continuous_update=False, description='entre $ \\eta $ em V  ($\\eta <…